## Multi layer RNN Language Model

This notebook only applies a slight modification to the previous notebook: use multilayer GRU instead of single layer

In [1]:
import torch
import torchtext
from torchtext import data

TEXT = data.Field(
    tokenizer_language='en',
    lower=True,
    init_token='<sos>',
    eos_token='<eos>',
    batch_first=True,
)

Now, we can load our dataset

In [2]:
from torchtext.datasets import WikiText2
 
train, valid, test = WikiText2.splits(TEXT) 

TEXT.build_vocab(train, vectors="glove.6B.300d")

print(f"We have {len(TEXT.vocab)} tokens in our vocabulary")

We have 28914 tokens in our vocabulary


## Iterator


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
BPTT_LEN = 30

train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=BATCH_SIZE,
    bptt_len=BPTT_LEN, # this is where we specify the sequence length
    device=device,
    repeat=False)

In [4]:
import torch.nn as nn

class RNNLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, pad_idx, hidden_size,
                 cell_class=nn.GRU, num_layers=1, dropout=0.20):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=PAD_IDX)
        self.rnn = cell_class(embedding_dim, hidden_size, 
                              batch_first=True, num_layers=num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    def forward(self, inp, hidden=None):
        """
        Inputs are supposed to be just one step (i.e. one letter)
        """
        # inputs = [batch_size, ]
        emb = self.embedding(inp)
        # emb = [batch, embedding_dim]
        # As all my examples are of the same length, there is no use 
        # in packing the input to the RNN
        rnn_outputs, hidden = self.rnn(emb, hidden)
        # hidden = [batch, hidden_dim]
        
        out = self.fc(self.dropout(rnn_outputs))
        # out = [batch, vocab size]

        return out, hidden

Create the Language Model. We will create a 3-layer RNN model.

In [5]:
import torch.optim as optim

HIDDEN_DIM = 512
vocab_size = TEXT.vocab.vectors.shape[0]
embedding_dim = TEXT.vocab.vectors.shape[1]

PAD_IDX = TEXT.vocab.stoi["<pad>"]
UNK_IDX = TEXT.vocab.stoi["<unk>"]
EOS_IDX = TEXT.vocab.stoi["<eos>"]
SOS_IDX = TEXT.vocab.stoi["<sos>"]


model = RNNLanguageModel(vocab_size, embedding_dim, hidden_size=HIDDEN_DIM, 
                         pad_idx=PAD_IDX, num_layers=2, dropout=0.4)

# Set weight for UNK to a random normal
model.embedding.weight.data.copy_(TEXT.vocab.vectors)
model.embedding.weight.data[UNK_IDX] = torch.randn(embedding_dim)

# Optimizer, criterion, and scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

model = model.to(device)
criterion = criterion.to(device)

## Training 

In [6]:
import torch
import numpy as np

def train(model, iterator, optimizer, criterion):
    """
    Trains the model for one full epoch
    """
    epoch_loss = 0
    epoch_perplexity = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()
        text = batch.text
        trg = batch.target.view(-1)
        
        preds, _ = model(text)
        preds = preds.view(-1, preds.shape[-1])
        
        loss = criterion(preds, trg)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_perplexity += np.exp(loss.item())
    
    return epoch_loss / len(iterator), epoch_perplexity / len(iterator)


def evaluate(model, iterator, criterion):
    """
    Evaluates the model on the given iterator
    """
    epoch_loss = .0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            trg = batch.target.view(-1)

            preds, _ = model(text)
            preds = preds.view(-1, preds.shape[-1])
            
            loss = criterion(preds, trg)

            epoch_loss += loss.item()
            
        loss = epoch_loss / len(iterator)
        
        perplexity = np.exp(loss)

    return loss, perplexity

In [7]:
from tqdm.notebook import tqdm
import time

N_EPOCHS = 30

best_valid_loss = float('inf')

early_stopping_tolerance = 5
epochs_without_improvement = 0

model_path = "/tmp/rnn_multilayer_lang_model.pt"

pbar = tqdm(range(N_EPOCHS), ncols=1000)
for epoch in pbar:
    
    epoch_bar = tqdm(train_iter)
    train_loss, train_perplexity = train(model, epoch_bar, optimizer, criterion)
    valid_loss, valid_perplexity = evaluate(model, valid_iter, criterion)
    lr_scheduler.step(valid_loss)
    
    desc = f' Train Loss: {train_loss:.3f} Perp: {train_perplexity:.2f}'
    desc += f' Val. Loss: {valid_loss:.3f} Perp: {valid_perplexity:.2f}'
    pbar.set_description(desc)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), model_path)
        print(f"Best model so far (Loss {best_valid_loss:.3f} Perp {valid_perplexity:.2f}) saved at {model_path}")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= early_stopping_tolerance:
            print("Early stopping")
            break


Best model so far (Loss 5.251 Perp 190.73) saved at /tmp/rnn_lang_model.pt



Best model so far (Loss 5.044 Perp 155.15) saved at /tmp/rnn_lang_model.pt



Best model so far (Loss 4.985 Perp 146.16) saved at /tmp/rnn_lang_model.pt



Best model so far (Loss 4.958 Perp 142.37) saved at /tmp/rnn_lang_model.pt



Early stopping


In [8]:
model.load_state_dict(torch.load(model_path))
model.eval()

valid_loss, valid_perplexity = evaluate(model, valid_iter, criterion)
test_loss, test_perplexity = evaluate(model, test_iter, criterion)


print(f"Valid loss      : {valid_loss:.2f}")
print(f"Valid perplexity: {valid_perplexity:.2f}\n")

print(f"Test loss      : {test_loss:.2f}")
print(f"Test perplexity: {test_perplexity:.2f}")

Valid loss      : 4.96
Valid perplexity: 142.37

Test loss      : 4.89
Test perplexity: 133.24


Well, it doesn't seem to be improving a lot the perplexity compared to the single layer model

## Sampling

In [9]:
import torch.nn.functional as F

def sample_sentence(init_token="<eos>", temperature=1):

    seq = [TEXT.vocab.stoi[init_token]]

    while len(seq) == 1 or seq[-1] != EOS_IDX:
        inp = torch.LongTensor([[seq[-1]]]).to(device)
        out, _ = model(inp)

        """
        Sample from probabilities
        """
        probs = F.softmax(out.view(-1) / temperature, dim=0)
        next_tok_idx = torch.multinomial(probs, num_samples=1)
        
        seq.append(next_tok_idx)
        
    return [TEXT.vocab.itos[t] for t in seq]

In [10]:
for temperature in np.arange(0.5, 1.5, 0.15):
    print("="*80, f"\nSampling with temperature = {temperature:.2f}")
    
    print(" ".join(sample_sentence("the", temperature=temperature)))

Sampling with temperature = 0.50
the <unk> , the <unk> , the second for the first one of <unk> , and the <unk> . the <unk> . <eos>
Sampling with temperature = 0.65
the novel , and the australian open . the <unk> , the only a night it is much of the original along with the underground and <unk> of the french and <unk> ( <unk> . <eos>
Sampling with temperature = 0.80
the company , which were produce a nearby in the bottom of jane = = = = = = = = <eos>
Sampling with temperature = 0.95
the old club football operations of these birds . a low being cushitic song . the loading deck and a extravagant emotions , it was observed for a months in the fear . <unk> claimed , the museum for the book sequences of all @-@ real reason for the 2005 2013 , <unk> ) ( 76 km ) with the new form <unk> , <unk> may occur . after the new united states , diverse older central south korean , nasa sports revealed that suffered more times enforcement . forty style . as evacuate pay to new male <unk> on the savage , 

As we rise temperature, we have more variety at the cost of meaningless stuff..

### Hidden State

There is a problem here! We are missing the hidden state

In [11]:
import torch.nn.functional as F

def sample_sentence(init_token="<eos>", temperature=1):

    seq = [TEXT.vocab.stoi[init_token]]
    hidden = None
    while len(seq) == 1 or seq[-1] != EOS_IDX:
        inp = torch.LongTensor([[seq[-1]]]).to(device)
        out, hidden = model(inp, hidden=hidden)

        """
        Sample from probabilities
        """
        probs = F.softmax(out.view(-1) / temperature, dim=0)
        next_tok_idx = torch.multinomial(probs, num_samples=1)
        
        seq.append(next_tok_idx)
        
    return [TEXT.vocab.itos[t] for t in seq]

In [12]:
for temperature in np.arange(0.5, 1.5, 0.15):
    print("="*80, f"\nSampling with temperature = {temperature:.2f}")
    
    print(" ".join(sample_sentence("the", temperature=temperature)))

Sampling with temperature = 0.50
the local government , a <unk> <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> <unk> , <unk> , <unk> , <unk> <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> <unk> , <unk> <unk> , <unk> , <unk> , <unk> , <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> <unk> , <unk> , <unk> <unk> , <unk> <unk> <unk> , <unk> <unk> <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> , <unk> <unk> , <unk

We can observe that:

- with hidden states there are more "meaningful" stuff
- quotation marks are closed when using the hidden state